In [ ]:
##########################################################################################################
# Author: Mihaly Sulyok & Peter Karacsonyi                                                               #
# Last updated: 2023 Dec 10                                                                              #
# This workbook does wsi preprocessing with Pathml version 2.1.1                                         #
# Input: wsi image files, tested with OpenSlide .tif                                                     #
# Transformations: MedianBlur, BinaryThreshold, Tiling, MorphOpen, MorphClose, ForegroundDetection       #
# Read more on transformations: https://pathml.readthedocs.io/en/latest/examples/link_gallery.html       #
# Output: h5path files (https://pathml.readthedocs.io/en/latest/h5path.html)                             #
##########################################################################################################

# local imports
import os
if os.name == 'nt':
    import helpers.openslideimport #on windows, openslide needs to be installed manually, check local openslideimport.py

# global imports
import time
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# pathml imports
from pathml import types
from pathml.core import SlideData, SlideDataset
from pathml.preprocessing import Pipeline, MorphOpen, MorphClose, BinaryThreshold

#########
# usage #
#########

#####################
# configure folders #
#####################

# set working directory
base_dir = Path("G:\\placenta")
# base_dir = Path("/mnt/g/xyz/") # linux
# wsi folder
wsi_subfolder = "wsi"
# place geojsons into this folder inside base with same name as the image (echino23.tiff / echino38.gejson)
annotations_subfolder = "none"
# output h5path files to (will be created)
h5path_subfolder = "h5"
# filenames having this substring with endpoint, possibly having annotation
endpoint_filename = "endpoint"

#######################################
# configure transformation parameters #
#######################################

pipeline = Pipeline([
    # values are the best found for placenta wsis
    BinaryThreshold(mask_name = "binary_threshold", use_otsu = True),
    MorphOpen(mask_name="binary_threshold", kernel_size=10, n_iterations=5),
    MorphClose(mask_name="binary_threshold", kernel_size=40, n_iterations=5),
])